<a href="https://colab.research.google.com/github/monjjjjj/Iris-Classification/blob/main/IrisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras import initializers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [42]:
#load data
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
x = dataset[:,0:4].astype('float32')
y = dataset[:,4]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=0)
#print(x.shape)
#print(len(x))
#print(train_x)
#print(len(train_x))

In [43]:
#fix random seed for reproducibility
seed = 9
numpy.random.seed(seed)

In [44]:
#encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
#print(encoded_y) 

#convert integers to one-hot vector
one_hot_y = np_utils.to_categorical(encoded_y)
#print(one_hot_y)

In [45]:
#build model
#4 inputs: sepal_length, sepal_width, petal_length, petal_width
#3 outputs: Iris Setosa[1, 0, 0], Versicolor[0, 1, 0], or Virginica[0, 0, 1]
def baseline_model():
  model = Sequential()
 
  #model.add(Conv1D(32, 3, activation='relu')(x.shape))
  #model.add(MaxPooling1D(2,data_format='channels_first'))
  #model.add(BatchNormalization())
  

  model.add(Dense(5, kernel_initializer='normal', input_dim=4, activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(3, kernel_initializer='normal', activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
  print(model.summary())
  return model

In [46]:
#training
model = baseline_model()
train_history = KerasClassifier(build_fn=baseline_model, nb_epoch=200, batch_size=5, verbose=0)


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 25        
                                                                 
 dense_1 (Dense)             (None, 6)                 36        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
#k-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(train_history, x, one_hot_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 5)                 25        
                                                                 
 dense_4 (Dense)             (None, 6)                 36        
                                                                 
 dense_5 (Dense)             (None, 3)                 21        
                                                                 
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 25        
                                                                 
 dense_7 (Dense)             

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 5)                 25        
                                                                 
 dense_19 (Dense)            (None, 6)                 36        
                                                                 
 dense_20 (Dense)            (None, 3)                 21        
                                                                 
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________
None


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 5)                 25        
                                                                 
 dense_22 (Dense)            (None, 6)                 36        
                                                                 
 dense_23 (Dense)            (None, 3)                 21        
                                                                 
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 5)                 25        
                                                                 
 dense_25 (Dense)            